In [1]:
import glob
import os
import subprocess
import librosa
import soundfile
import librosa.display

import soundfile as sf
import numpy as np
from matplotlib import pyplot as plt

DIR_m4a = './soundJapanese/test'
DIR_wav = './soundJapanese/test2'

filename_m4a = glob.glob(DIR_m4a + '/*.m4a')

fileNum = sum(os.path.isfile(os.path.join(DIR_m4a, name)) for name in os.listdir(DIR_m4a))-1

for num in range(fileNum):
    basename_without_ext = os.path.splitext(os.path.basename(filename_m4a[num]))[0]
    newname = DIR_wav + '/%s.wav' % basename_without_ext
    cmd = 'ffmpeg -i %s -f wav %s' %(filename_m4a[num], newname)
    subprocess.run(cmd, shell=True)

In [3]:
# data, samplerate = sf.read(newname)
data, samplerate = sf.read('./soundJapanese/test2/a3.wav')
t = np.arange(0, len(data))/samplerate
b = abs(data) > 0.025#閾値仮置き

# plt.figure(figsize=(18, 6))
# plt.plot(t,abs(data))
# plt.plot(t, b)
# plt.show()

#xと一致した要素が配列の何番目かを取得
def my_index_multi(l, x):
    return [i for i, _x in enumerate(l) if _x == x]

high_num = my_index_multi(b, 1)
high_num_first = high_num[0]
high_num_last = high_num[-1]#配列の最後を取り出す

data_cut = data[high_num_first:high_num_last]

soundfile.write(DIR_wav + '/1_J2.wav', data_cut, samplerate)

In [4]:
#y, sr = librosa.load(DIR_wav + '/1_J.wav', sr=16000) # y is a numpy array of the wav file, sr = sample rate
#y_shifted = librosa.effects.pitch_shift(y, sr, n_steps=2) # shifted by 4 half steps

In [17]:
def sort_rename(rootdir):
    xs = []
    for root, dir, files in os.walk(rootdir):
        for f in files:
            path = os.path.join(root, f)
            xs.append((os.path.getmtime(path), path))

    count = 0
    for mtime, path in sorted(xs):
        count = count + 1
        name = os.path.basename(path)
        t = datetime.datetime.fromtimestamp(mtime)
        os.rename(rootdir + '/' + name,rootdir + '/a' +  '{0:04d}'.format(count) + '.m4a')
        print('{0:02d}'.format(3))
        # => 03
        print(t, name)
        
sort_rename('./soundJapanese/test')

03
2021-01-15 07:05:33 a001.m4a
03
2021-01-15 07:05:33 a002.m4a
03
2021-01-15 07:05:45 a003.m4a
03
2021-01-15 07:05:45 a004.m4a
03
2021-01-15 07:05:50 a005.m4a
03
2021-01-15 07:05:50 a006.m4a
03
2021-01-17 18:41:54 a007.m4a
03
2021-01-17 18:42:05 a008.m4a
03
2021-01-17 18:42:17 a009.m4a
03
2021-01-17 18:43:10.067213 a010.m4a


In [33]:
#初期設定

import glob
import os
import subprocess
import librosa
import soundfile
import datetime
import librosa.display

import soundfile as sf
import numpy as np
from matplotlib import pyplot as plt

DIR_m4a = './soundJapanese/test3'
DIR_wav = './soundJapanese/test2'
DIR_wav_change = './soundJapanese/test4'

#xと一致した要素が配列の何番目かを取得
def my_index_multi(l, x):
    return [i for i, _x in enumerate(l) if _x == x]

In [34]:
#ファイルの名前づけ

xs = []
for root, dir, files in os.walk(DIR_m4a):
    for f in files:
        path = os.path.join(root, f)
        xs.append((os.path.getmtime(path), path))

count = 0
for mtime, path in sorted(xs):
    count = count + 1
    name = os.path.basename(path)
    t = datetime.datetime.fromtimestamp(mtime)
    os.rename(DIR_m4a + '/' + name,DIR_m4a + '/a' +  '{0:04d}'.format(count) + '.m4a')

In [37]:
#mp4をwavに変換

filename_m4a = glob.glob(DIR_m4a + '/*.m4a')

fileNum = sum(os.path.isfile(os.path.join(DIR_m4a, name)) for name in os.listdir(DIR_m4a))

for num in range(fileNum):
    basename_without_ext = os.path.splitext(os.path.basename(filename_m4a[num]))[0]
    newname = DIR_wav + '/%s.wav' % basename_without_ext
    cmd = 'ffmpeg -i %s -f wav %s' %(filename_m4a[num], newname)
    subprocess.run(cmd, shell=True)
    
    #音声を変換
    data, samplerate = sf.read(newname)
    t = np.arange(0, len(data))/samplerate
    b = abs(data) > 0.025 #閾値仮置き→閾値より大きい場合1,小さい場合0

    high_num = my_index_multi(b, 1) #閾値を超えたデータを抜き出し
    high_num_first = high_num[0]
    high_num_last = high_num[-1]#配列の最後を取り出す

    data_cut = data[high_num_first:high_num_last]
    
    soundfile.write(DIR_wav_change + '/%s_change.wav' %(basename_without_ext), data_cut, samplerate)

['./soundJapanese/test3/a0009.m4a', './soundJapanese/test3/a0008.m4a', './soundJapanese/test3/a0005.m4a', './soundJapanese/test3/a0004.m4a', './soundJapanese/test3/a0006.m4a', './soundJapanese/test3/a0007.m4a', './soundJapanese/test3/a0003.m4a', './soundJapanese/test3/a0002.m4a', './soundJapanese/test3/a0001.m4a']
